In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import acquire
import prepare
import geopy
from geopy.geocoders import Nominatim
import re
from datetime import timedelta, datetime

%matplotlib inline

In [ ]:
url_data_dict_311 = 'https://data.sanantonio.gov/dataset/93b0e7ee-3a55-4aa9-b27b-d1817e91aec3/resource/0eab0cfc-a114-4064-9f25-d088d8ac46a3/download/datadictionary-311.xlsx'
data_dictionary_311_new = pd.read_excel(url_data_dict_311, header=2)

In [2]:
df_311 = acquire.acquire_311()

In [ ]:
df_weather = acquire.acquire_weather()
df_weather.sample(19)

In [ ]:
df_sso = acquire.acquire_sso()

## Prepare

## Get 311 for prepare

In [3]:
df_311 = prepare.prepare_311(df_311)

In [ ]:
df_311.category.value_counts()

In [4]:
df_311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 571737 entries, 0 to 571736
Data columns (total 19 columns):
 #   Column                          Non-Null Count   Dtype          
---  ------                          --------------   -----          
 0   category                        571737 non-null  object         
 1   case_id                         571737 non-null  int64          
 2   late                            571737 non-null  object         
 3   dept                            529331 non-null  object         
 4   reason_name                     571737 non-null  object         
 5   event_name                      571737 non-null  object         
 6   case_status                     571737 non-null  object         
 7   sourceid                        571737 non-null  object         
 8   event_address                   571737 non-null  object         
 9   council_district                571737 non-null  int64          
 10  report_start_date               571737 non-n

In [5]:
df_311.shape

(571737, 19)

In [ ]:
df_sso.head(3)

In [ ]:
df_sso['full_address'] = df_sso.SPILL_ST_NAME.astype(str) + ', San Antonio, Texas'

In [ ]:
df_311.case_days_length.plot()

In [7]:
df_311.zip_code.isna().sum()

89473

In [53]:
df_311.head()

,category,case_id,late,dept,reason_name,event_name,case_status,sourceid,event_address,council_district,report_start_date,report_ending_date,zip_code,reported_date,closed_date,due_date,case_days_length,report_to_assigned_days_length,days_past_due_length
0,Streets & Infrastructure,1010126853,YES,Trans & Cap Improvements,Storm Water,Tree Fallen,Closed,Constituent Call,ELM CREEK and WURZBACH RD,8,6/13/2019,6/13/2020,NaN,2011-12-10,2020-03-05 00:00:00.000000,2011-12-11,3008 days 00:00:00,1 days,3007 days 00:00:00
1,Streets & Infrastructure,1010126880,YES,Trans & Cap Improvements,Streets,Barricade - Misc. Special Event/Emergency,Closed,Constituent Call,"1600 CENTENNIAL BV, San Antonio, 78211",5,6/13/2019,6/13/2020,78211,2011-12-10,2020-03-05 00:00:00.000000,2011-12-13,3008 days 00:00:00,3 days,3005 days 00:00:00
2,Information,1010338011,NO,Knowledge Management,Knowledge,Knowledge Solution Extra,Closed,Constituent Call,"310 ROOSEVELT AV, San Antonio, 78210",5,6/13/2019,6/13/2020,78210,2012-05-25,2020-01-27 00:00:00.000000,2012-05-25,2803 days 00:00:00,0 days,2803 days 00:00:00
3,Graffiti,1010444245,YES,Code Enforcement Services,Graffiti,Graffiti Public Property,Open,Web Portal,"600 NOGALITOS ST, San Antonio, 78204",5,6/13/2019,6/13/2020,78204,2012-08-15,2020-06-21 12:41:39.612820,2012-08-30,2867 days 12:41:39.612820,15 days,2852 days 12:41:39.612820
4,Property Maintenance,1010888252,YES,Code Enforcement Services,Code Enforcement (IntExp),Alley-Way Maintenance,Open,Web Portal,"6043 CASTLE QUEEN, San Antonio, 78218",2,6/13/2019,6/13/2020,78218,2013-06-06,2020-06-21 12:41:39.612820,2013-08-09,2572 days 12:41:39.612820,64 days,2508 days 12:41:39.612820


In [8]:
ls

Stops_LatLong_GeocodeData_2020.csv  prepare.py
__pycache__/                        scratch.ipynb
acquire.py                          sso_data.csv
data_311.csv                        weather_data.csv


In [65]:
lat_n_lon = pd.read_csv('Stops_LatLong_GeocodeData_2020.csv')

In [66]:
lat_n_lon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9859 entries, 0 to 9858
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   StopNumber    9859 non-null   object 
 1   Location      9859 non-null   object 
 2   StreetNumber  9715 non-null   float64
 3   Street        9854 non-null   object 
 4   City          9854 non-null   object 
 5   State         9854 non-null   object 
 6   Zip           9854 non-null   float64
dtypes: float64(2), object(5)
memory usage: 539.3+ KB


In [67]:
(lat_n_lon.StreetNumber.isna() == True).sum()

144

In [68]:
lat_n_lon['Zip'] = lat_n_lon.Zip.fillna(0)

In [69]:
lat_n_lon.shape

(9859, 7)

In [70]:
lat_n_lon.drop(columns=(['StopNumber', 'StreetNumber']), inplace=True)

In [71]:
lat_n_lon['Location'] = lat_n_lon.Location.str.replace('&', 'and')

In [72]:
lat_n_lon.sample(10)

,Location,Street,City,State,Zip
6655,HENDERSON PASS BTW.1000 OAKS/R OAKS,Thousand Oaks Dr,San Antonio,TX,78232.0
938,CLASSEN RD. AND MISTY GLADE,Dulce Creek Dr,San Antonio,TX,78247.0
2287,LOOP 410 N ACCESS RD. and HWY. 151,I 410,San Antonio,TX,78238.0
7771,BLANCO AND WEST AVE.,Blanco Rd,San Antonio,TX,78216.0
5664,IH-35 N ACCESS RD. and EVANS RD.,N Evans Rd,Selma,TX,78154.0
5348,EISENHAUER AND CHEVY CHASE,Bethencourt,San Antonio,TX,78209.0
2030,BRIGHT VALLEY IN FRONT OF 6103,Bright Vly,San Antonio,TX,78242.0
7700,SAN PEDRO and ELMIRA,San Pedro Ave,San Antonio,TX,78205.0
329,BROADWAY IN FRONT OF ATandT BLDG.,Broadway,San Antonio,TX,78209.0
6021,WORTH AND STANLEY,Stanley Rd,Jbsa Ft Sam Houston,TX,78234.0


In [73]:
lat_n_lon.head()

,Location,Street,City,State,Zip
0,ST. MARY'S BETWEEN CONVENT and MART,N St Mary's St,San Antonio,TX,78205.0
1,ST. MARY'S and PECAN,E Pecan St,San Antonio,TX,78205.0
2,ST. MARY'S BETWEEN TRAVIS and HOUSTON,E Houston St,San Antonio,TX,78205.0
3,ST. MARY'S and COMMERCE,S St Mary's St,San Antonio,TX,78205.0
4,COMMERCE and SOLEDAD,Main Plz,San Antonio,TX,78205.0


In [75]:
lat_n_lon['zip_code'] = lat_n_lon.Zip.astype('int')

In [79]:
lat_n_lon.sample(5)

,Location,Street,City,State,Zip,zip_code
6627,CESAR CHAVEZ FRNT OF J.H. WOOD BLDG,E Cesar Chavez Blvd,San Antonio,TX,78205.0,78205
7402,DOLLARHIDE and METZ,Metz Ave,San Antonio,TX,78223.0,78223
5831,S. NEW BRAUNFELS and BETTY JEAN,Betty Jean,San Antonio,TX,78223.0,78223
4746,HACKAMORE and WESTFIELD,Westfield Dr,San Antonio,TX,78227.0,78227
3064,HILLSIDE AND EL PRADO W.,W El Prado,San Antonio,TX,78212.0,78212


In [82]:
(df_311.zip_code.isna() == True).sample(25)

61039     False
434039     True
27227     False
531772    False
362683    False
195493    False
205297    False
188233    False
76144     False
271970    False
459144    False
302547    False
368744    False
427934    False
200883     True
29419     False
538115     True
100934    False
484605    False
11192     False
405911    False
58673      True
120158    False
556271     True
509083     True
Name: zip_code, dtype: bool